# Introduction to Dask and the Open Data Cube <img align="right" src="../../resources/csiro_easi_logo.png">

Prerequisites: This material assumes basic knowledge of the Open Data Cube, Xarray and numerical processing using numpy.

The Open Data Cube library is written in Python and makes extensive use of scientific and geospatial libraries.
For the purposes of this tutorial we will primarily consider five libraries:

  1. `datacube` - EO datacube
  1. `xarray` - labelled arrays
  1. (optional) `dask` & `distributed` - distributed parallel programming
  1. `numpy` - numerical array processing with vectorisation
  1. (optional) `numba` - a library for high performance python

Whilst the interrelations are intimate it is useful to conceptualise them according to their primary role and how these roles build from low level numerical array processing (`numpy`) through to high-level EO datacube semantics (`datacube` and `xarray`). If you prefer, viewed from top to bottom we can say:
  1. `datacube.load()` does the necessary file IO and data manipulation to construct a...
  1. `xarray` which will be labelled with the necessary coordinate systems and band names and made up of...
  1. (optionally) `dask.array`s which contain many `chunks` which are...
  1. `numpy` arrays containing the actual data values.

Each higher level of abstraction thus builds on the lower level components that perform the actual storage and computation.

Overlaid on this are libraries like `numba`, `dask` and `distributed` that provide computational components that can accelerate and distribute processing across multiple compute cores and computers. The use of `dask`, `distributed` and `numba` are optional - not all applications require the additional complexity of these tools.

Given these relationships it is clear that achieving performance and scale requires an understanding of the performance of each library and how it interacts with the others. Moreover, and often counterintuitively, adding more compute cores to a problem may not make it faster, in fact it may slow down (as well as waste resources). Added to that is the _deceptive simplicity_ in that some of the tools can be simply _turned on_ with only a few code changes, and little knowledge, and significant performance increases can be achieved.

As the application is then scaled, or an alternative algorithm used, further challenges may ensue that require major refactors and changes in algorithmic approach, undoing some of the earlier work and often leading to great frustration. The good news is whilst there clearly is complexity (six interrelated libraries mentioned so far), there are common concepts and techniques involved in analysing _how to optimise your algorithm_. If you know from the start your application is going to require scale, then it does help to think in advance where you are heading.

This course will equip readers with concepts and techniques they can utilise in their algorithm and workflow development. The course will be using computer science terms and a variety of libraries but won't be discussing these in detail in order to keep this course concise. The focus will be on demonstration by example and analysis techniques to identify where to focus effort. The reader is encouraged to use their favorite search engine to dig deeper when needed; there are a lot of tutorials online!

One last thing, in order to maintain a healthy state of mind for "Dask and ODC", the reader is encouraged to hold both of these truths in mind at the same time:
  1. The *best* thing about dask is it makes distributed parallel programming in the datacube easy
  1. The *worst* thing about dask it is makes distributed parallel programming in the datacube easy

Yep, that's contradictory! By the end of this course, and a couple of your own adventures, you will understand why.

In [ ]:
import git
import sys
import os
os.environ['USE_PYGEOS'] = '0'
repo = git.Repo('.', search_parent_directories=True)
if repo.working_tree_dir not in sys.path: sys.path.append(repo.working_tree_dir)
from easi_tools import EasiNotebooks, notebook_utils

In [ ]:
easi = EasiNotebooks()

# Performance in Python

In this section we'll explore python performance when doing array processing. Python itself, as you will soon see, is quite slow. It is, however, highly expressive and can orchestrate more complex and faster libraries of numerical code (e.g. `numpy`). Python is also ammendable to being accelerated (e.g. using `numba`) and made to run on multiple CPU cores (e.g. via `dask`). 

## Python `list` and `numpy`

Let's take a look at the simple addition of two arrays. In Python the nearest data type to an array is a `list` of numbers. This will be our starting point.

Our focus is on performance so we'll use the Jupyter `%%time` and `%%timeit` magics to run our cells and time their execution. The latter will run the cell multiple times and provide us with more representative statistics of performance and variability.

First in pure Python using lists :

In [ ]:
size_of_vec = 2000*2000
X_list = range(size_of_vec)
Y_list = range(size_of_vec)

In [ ]:
%%timeit -n 10 -r 1
Z = [X_list[i] + Y_list[i] for i in range(len(X_list)) ]

Now the same processing using `numpy`.

In [ ]:
import numpy
X = numpy.arange(size_of_vec)
Y = numpy.arange(size_of_vec)

In [ ]:
%%timeit -n 10 -r 1
Z = X + Y

Let's check that the two arrays are identical (note that %%timeit does not make the variables above available due to the way that %%timeit works, so we reconstruct the arrays).

In [ ]:
print("Are the arrays identical?")
([X_list[i] + Y_list[i] for i in range(len(X_list)) ] == X + Y).all()

At least two orders of magnitude in performance improvement!

Why?

`numpy` provides a python interface to an underlying C array library that makes use of CPU `vectorization` - this allows it to process several add operations at the same time.

`numpy` isn't the only libray that does this type of wrapping over a fast optimised library. There is `cuPy` which uses GPUs for array processing. `tensorflow` uses both CPU and GPU optimisations for machine learning. `datashader` for large dataset visualisation...
It's a very long list and thanks to a great deal of work by a great many software engineers most of these libraries will work together efficiently. 

> __Tip__: Where possible use high performance libraries with python wrappers for performance

The reader will have noticed the change in abstraction. The pure Python version used list comprehension syntax to add the two arrays, `numpy` was a much shorter direct addition syntax, much more in keeping with the mathematics involved. This change in abstraction is seen in most libraries, including the ODC library where `datacube.load()` is shorthand for a complex process of data discovery, reprojection, fusing and array construction. High-level abstractions like this are powerful and greatly simplify development (the good). They can also hide performance bottlenecks and challenges (the bad).

> __Tip__: Use high level API abstractions but be mindful of their use

## `Numba` - accelerating Python

So high performance libraries rock, but what if you don't have one for your purpose and you're back in Python?
`Numba` translates Python functions into optimized machine code at runtime - https://numba.pydata.org.

Let's see how this works. A more complex example this time with a smoothing function applied over our (random) image, perform an FFT, and save the result.
These examples are (very) slightly modified versions from the High Performance Python Processing Pipeline video by Matthew Rocklin: https://youtu.be/wANQkgDuTAk
It's such a good introduction its worth repeating.

We'll also use the fantastic `tqdm` to provide a progress bar.

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

def load_eo_data():
    return np.random.random((1000, 1000))

def smooth(x):
    out = np.empty_like(x)
    for i in range(1, x.shape[0] - 1):
        for j in range(1, x.shape[1] - 1):
            out[i, j] = (x[i + -1, j + -1] + x[i + -1, j + 0] + x[i + -1, j + 1] +
                         x[i +  0, j + -1] + x[i +  0, j + 0] + x[i +  0, j + 1] +
                         x[i +  1, j + -1] + x[i +  1, j + 0] + x[i +  1, j + 1]) // 9

    return out

def save(x, filename):
    pass
        

In [ ]:
%%time
for i in tqdm(range(5)):
    img = load_eo_data()
    img = smooth(img)
    img = np.fft.fft2(img)
    save(img, "file-" + str(i) + "-.dat")

The `smooth(x)` function contains two python loops. Now we could (and would) find a similar high performance library with a `smooth(x)` function but for this example let's use `numba`'s `jit` compiler to translate the python function into optimized machine code at runtime.

In [ ]:
import numba

fast_smooth  = numba.jit(smooth)

In [ ]:
%%time

for i in tqdm(range(5)):
    img = load_eo_data()
    img = fast_smooth(img)
    img = np.fft.fft2(img)
    save(img, "file-" + str(i) + "-.dat")

Just a bit quicker! Much of the time in the first run was `numba` performing compilation. Run the cell above again and you'll find it runs faster the second time.

The _recommended_ approach to have `numba` compile a python function is to use python decorator syntax (`@numba.jit`). So the original code now looks like this (single line changed) and we can call `smooth(x)` without having to create `fast_smooth`:

In [ ]:
import numpy as np

def load_eo_data():
    return np.random.random((1000, 1000))

@numba.jit
def smooth(x):
    out = np.empty_like(x)
    for i in range(1, x.shape[0] - 1):
        for j in range(1, x.shape[1] - 1):
            out[i, j] = (x[i + -1, j + -1] + x[i + -1, j + 0] + x[i + -1, j + 1] +
                         x[i +  0, j + -1] + x[i +  0, j + 0] + x[i +  0, j + 1] +
                         x[i +  1, j + -1] + x[i +  1, j + 0] + x[i +  1, j + 1]) // 9

    return out

def save(x, filename):
    pass

In [ ]:
%%time
for i in tqdm(range(5)):
    img = load_eo_data()
    img = smooth(img)
    img = np.fft.fft2(img)
    save(img, "file-" + str(i) + "-.dat")

Why not use `numba` all the time everywhere?

Like most high level abstractions `numba` makes assumption about code, only accelerates a subset of python libraries (not all `numpy` functions are available via `numba`), and it is entirely possible it can make performance worse or not work at all!

There's one additional consideration. If you've run all the cells to this point in order, try running the `fast_smooth` cell again, repeated below for convenience, just run this:


In [ ]:
fast_smooth  = numba.jit(smooth)

Error!

The `smooth` function was decorated so the second time is already `jit` compiled. Attempting to do so again causes this error, and exposes some of the low level changes behind the abstraction.
This can make debugging code difficult if you are not mindful of what is occuring.

TIP: __Use high level API abstractions but be mindful of their use__

# Parallelism with Dask

Our fake EO processing pipeline only has 5 images and takes about 1 sec to run. In practice we'll have 1000s of images to process (if not more).

Let's repeat our example code but now with more iterations. You can understand why we use The `tqdm` library to provide a progress bar for these larger scale examples rather than printing out each iteration number or staring at a blank screen wondering if it works!


In [ ]:
import numpy as np
import numba
from tqdm.notebook import tqdm


def load_eo_data():
    return np.random.random((1000, 1000))

@numba.jit
def smooth(x):
    out = np.empty_like(x)
    for i in range(1, x.shape[0] - 1):
        for j in range(1, x.shape[1] - 1):
            out[i, j] = (x[i + -1, j + -1] + x[i + -1, j + 0] + x[i + -1, j + 1] +
                         x[i +  0, j + -1] + x[i +  0, j + 0] + x[i +  0, j + 1] +
                         x[i +  1, j + -1] + x[i +  1, j + 0] + x[i +  1, j + 1]) // 9

    return out

def save(x, filename):
    pass

Before running the next code, open a terminal window (File>New>Terminal) and run `htop` at the command line to show current CPU usage per core.
> **TIP:** Drag your terminal window so that it sits below this notebook before your un `htop` to see both windows at the same time.

In [ ]:
%%time
for i in tqdm(range(1000)):
    img = load_eo_data()
    img = smooth(img)
    img = np.fft.fft2(img)
    save(img, "file-" + str(i) + "-.dat")

You'll notice that only one core is showing any load. The above code is not using any of the additional cores.

`dask` can be useful in this scenario even on a local machine. 

Firstly, a few notes on terminology. A Dask Cluster is comprised of a __client__, a __scheduler__, and __workers__. These terms will be used throughout this tutorial. Figure 1 below shows the relationship between each of these components. The __client__ submits tasks to the __scheduler__, which decides how to submit the tasks to individual workers. During this process, the scheduler creates what is called a __Task Graph__. This is essentially a map of the tasks that need to be carried out. Figure 2 shows an example of a simple task graph (see https://docs.dask.org/en/stable/graphs.html for more information. __Workers__ carry out the actual calculations and either store the results or send them back to the client.

<div>
    <span style="border:solid 1px #888;float:left;padding:10px;margin-right:25px;width:550px">
        <img src="../../resources/distributed-overview.png">
        <figcaption><em>Figure 1. Overview of a Dask Cluster.</em></figcaption>
    </span>
    <span style="border:solid 1px #888;float:left;padding:10px;margin-left:25px;width:150px">
        <img style="float:left" src="../../resources/dask-simple.png">
        <figcaption><em>Figure 2. A simple Task Graph.</em></figcaption>
    </span>
</div>

Dask has several core data types, including __Dask DataFrames__ and __Dask Arrays__. Essentially, Dask DataFrames are parallelized Pandas DataFrames (Figure 3) and Dask Arrays are parallelized Numpy arrays (Figure 4).

<div style="width:100%">
    <span style="border:solid 1px #888;float:left;padding:10px;margin-right:25px;width:200px">
        <img src="../../resources/dask-dataframe.svg">
        <figcaption><em>Figure 3. A Dask DataFrame is comprised of many in-memory pandas DataFrames separated along an index.</em></figcaption>
    </span>
    <span style="border:solid 1px #888;float:left;padding:10px;margin-left:25px;width:300px">
        <img style="float:left" src="../../resources/dask-array.svg">
        <figcaption><em>Figure 4. A Dask Array is a subset of the NumPy <code>ndarray</code> interface using blocked algorithms, cutting up the large array into many small arrays.</em></figcaption>
    </span>
</div>

EASI and the Open Data Cube primarily make use of __Dask DataArrays__. For more information see https://tutorial.dask.org/02_array.html.

Let's start by creating a local dask cluster:

In [ ]:
from dask.distributed import Client, LocalCluster, fire_and_forget, wait

cluster = LocalCluster()
client = Client(cluster)
client

There is also a utility function in `notebook_utils` which you can use to initialize a local dask cluster. This can also be used to initialize a remote cluster using Dask Gateway, but pleaes complete the other Dask tutorials before using a remote cluster.

The following lines will initialize and display a local dask cluster and can replace the code above.

```python
cluster, client = notebook_utils.initialize_dask(use_gateway=False, wait=True)
display(cluster if cluster else client)
```

The Dask Dashboard url will show as "localhost" or "127.0.0.1" since its running locally in the Jupyter kernel. It can accessed using the Jupyter server proxy via the following url:

In [ ]:
notebook_utils.localcluster_dashboard(client=client,server=easi.hub)

You will want to have this open when running the next cell.

To `submit` the functions to run on the dask cluster is a straightforward modification to the syntax:

In [ ]:
%%time

for i in tqdm(range(1000)):
    img = client.submit(load_eo_data, pure=False)
    img = client.submit(smooth, img)
    img = client.submit(np.fft.fft2, img)
    future = client.submit(save, img, "file-" + str(i) + "-.dat")
    fire_and_forget(future)

If you watch `htop` in the terminal you'll see all cores become active. The dask dashboard will also provide a view of the tasks being run in parallel.

You will also see that the cluster remains busy after the cell above finishes. This is because Dask is working in the background processing the tasks which have been submitted to it in parallel.

A `dask.distributed.LocalCluster()` will shutdown when this notebook kernel is stopped.
Still it's a good practice to close the client and the cluster so its all cleaned up.This will be more important when using dask distributed clusters as they are independent of the notebook kernel.

In [ ]:
client.close()

cluster.close()